<a href="https://colab.research.google.com/github/Mojtaba-Choopani/huggingface-llm-course-fa-notebooks/blob/main/chapter2-USING-TRANSFORMERS/section5-From-tokenizer-to-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">

<b style="font-size: 18px;"> جمع‌بندی توکنایزر</b>
<div>

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

<div dir="rtl">

<p>
در بخش‌های اخیر، سعی کردیم بیشتر کارها را به‌صورت دستی انجام دهیم. نحوه عملکرد توکنایزرها را بررسی کردیم و به <b>توکن‌سازی</b>، <b>تبدیل به IDهای ورودی</b>، <b>پدگذاری</b>، <b>کوتاه‌سازی</b>، و <b>مکانیزم توجه</b> پرداختیم.
</p>

<p>
با این حال، همان‌طور که در بخش ۲ دیدیم، <b>API Transformers از </b> می‌تواند همه این‌ها را برای ما با یک تابع سطح بالا انجام دهد که در اینجا به آن می‌پردازیم.
</p>

<p>
وقتی توکنایزر را مستقیماً روی جمله فراخوانی می‌کنید، ورودی‌هایی دریافت می‌کنید که <b>آماده ارسال به مدل شما هستند</b>:
</p>

</div>


In [5]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs)

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs)

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


<div dir="rtl">

<p>
این API همچنین می‌تواند <b>چندین دنباله</b> را به‌طور هم‌زمان پردازش کند، بدون اینکه نیازی به تغییر در API باشد:
</p>

</div>


In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

<div dir="rtl">

<p>
این API می‌تواند بر اساس <b>چندین هدف مختلف</b> <b>پدگذاری (padding)</b> انجام دهد:
</p>

</div>


In [ ]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

<div dir="rtl">

<p>
این API همچنین می‌تواند <b>دنباله‌ها را کوتاه کند</b>:
</p>

</div>


In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

<div dir="rtl">

<p>
توکنایزر می‌تواند ورودی‌ها را به <b>تنسورهای فریم‌ورک‌های مختلف</b> تبدیل کند، مانند تنسورهای PyTorch ("<code>pt</code>") یا آرایه‌های NumPy ("<code>np</code>")، که سپس می‌توانند مستقیماً به مدل ارسال شوند.
</p>

</div>


In [8]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")
print(model_inputs)

{'input_ids': array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  2061,  2031,  1045,   999,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


<div dir="rtl">

<b style="font-size: 18px;"> توکن‌های ویژه</b>

<p>
اگر به <b>IDهای ورودی</b> بازگشتی از توکنایزر نگاه کنیم، متوجه خواهیم شد که کمی با آنچه قبلاً داشتیم متفاوت هستند:
</p>

</div>


In [9]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


<div dir="rtl">

<p>
توکنایزر به‌طور خودکار <b>توکن‌های ویژه</b> <code>[CLS]</code> و <code>[SEP]</code> را به ابتدای و انتهای جمله اضافه می‌کند، چون مدل‌ها برای پیش‌بینی به این توکن‌ها نیاز دارند. این کار برای هر مدل به‌طور خودکار انجام می‌شود.
</p>

</div>


In [10]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


<div dir="rtl">

<b style="font-size: 18px;"> جمع‌بندی: از توکنایزر تا مدل</b>

<p>
توکنایزر می‌تواند به‌طور خودکار <b>چندین دنباله را پدگذاری کند</b>، <b>دنباله‌های بلند را کوتاه کند</b> و <b>انواع مختلف تنسورها را با استفاده از API اصلی خود پردازش کند</b>.
</p>

</div>


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
